<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux :
   - les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest* ;
   - apparition de mots indicateurs dans le voisinage de *interest*.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de la page).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'on considère aussi les occurrences au pluriel (*interests*) ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

In [1]:
# Veuillez répondre ici (en commentaire) aux questions.
# A. 
# https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz et https://www.d.umn.edu/~tpederse/Data/README.int.txt

# B.
# Il contient 2368 phrases contenant au moins une occurence du mot `interest`,
# séparées ligne par ligne par le symbole '$$'

# C. 
# Oui

# D.
# L'occurence supplémentaire est annotée comme `*interest`

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [2]:
# Veuillez répondre ici (en commentaire) à la question.
# Les occurences du mot `interest` sont  annotées de 1 à 6 en fonction du sens comme suit:
# Sense 1 =  361 occurrences (15%) - readiness to give attention
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
# Sense 5 =  500 occurrences (21%) - a share in a company or business
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [3]:
# Veuillez répondre ici (en commentaire) à la question.
# Le 'Longman Dictionary of Contemporary English'
# Sense 1 = if you have an interest in something or someone, you want to know or learn more about them
# Sense 2 = a quality or feature of something that attracts your attention or makes you want to know more about it
# Sense 3 = an activity that you enjoy doing or a subject that you enjoy studying
# Sense 4 = the things that bring advantages to someone or something
# Sense 5 = if you have an interest in a particular company or industry, you own shares in it
# Sense 6 = the extra money that you must pay back when you borrow money

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [4]:
# Veuillez répondre ici (en commentaire) à la question.
# WordNet contient les 7 synsets suivant:
# Synsets 1 = a sense of concern with and curiosity about someone or something
# Synsets 2 = a reason for wanting something done
# Synsets 3 = the power of attracting or holding one's attention (because it is unusual or exciting etc.)
# Synsets 4 = a fixed charge for borrowing money; usually a percentage of the amount borrowed
# Synsets 5 = (law) a right or legal share of something; a financial involvement with something
# Synsets 6 = (usually plural) a social group whose members control some field of activity and who have common aims
# Synsets 7 = a diversion that occupies one's time and thoughts (usually pleasantly)

# On peut établir la correspondance suivante:
# Sense 1 = Synset 1
# Sense 2 = Synset 3
# Sense 3 = Synset 7
# Sense 4 = Synset 2
# Sense 5 = Synset 5
# Sense 6 = Synset 4

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [5]:
import nltk
import random
import re

In [6]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes)
readme_senses = [
    "readiness to give attention",
    "quality of causing attention to be given to",
    "activity, etc. that one gives attention to",
    "advantage, advancement or favor",
    "a share in a company or business",
    "money paid for the use of money"
]

senses1 = [list(set(filter(lambda s: len(s) >= 4, s.split()))) for s in readme_senses]

print(senses1)

[['give', 'readiness', 'attention'], ['causing', 'attention', 'given', 'quality'], ['that', 'etc.', 'activity,', 'attention', 'gives'], ['advantage,', 'advancement', 'favor'], ['company', 'business', 'share'], ['paid', 'money']]


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [7]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
ldoce_senses = [
    "if you have an interest in something or someone, you want to know or learn more about them",
    "a quality or feature of something that attracts your attention or makes you want to know more about it",
    "an activity that you enjoy doing or a subject that you enjoy studying",
    "the things that bring advantages to someone or something",
    "if you have an interest in a particular company or industry, you own shares in it",
    "the extra money that you must pay back when you borrow money"
]

senses2 = [list(set(filter(lambda s: len(s) >= 4, s.split()))) for s in map(lambda t: t[0] + " " + t[1], zip(readme_senses, ldoce_senses))]

print(senses2)

[['something', 'someone,', 'learn', 'more', 'them', 'readiness', 'attention', 'interest', 'know', 'have', 'about', 'give', 'want'], ['that', 'feature', 'causing', 'given', 'quality', 'your', 'more', 'attention', 'know', 'attracts', 'something', 'about', 'makes', 'want'], ['that', 'etc.', 'activity,', 'activity', 'studying', 'doing', 'attention', 'subject', 'enjoy', 'gives'], ['favor', 'that', 'bring', 'something', 'advantage,', 'things', 'someone', 'advantages', 'advancement'], ['particular', 'industry,', 'company', 'shares', 'interest', 'share', 'have', 'business'], ['when', 'that', 'back', 'borrow', 'must', 'paid', 'money', 'extra']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [8]:
def load_interest(filename):
    TO_IGNORE = ["$", "=", ".", "'", ",", "\n", "`"]
    ret = []
    with open(filename, "r") as f:
        for line in f.readlines():
            for t in TO_IGNORE:
                line = line.replace(t, "")
            if line != "":
                ret.append(nltk.tokenize.word_tokenize(line))
    return ret

In [9]:
# Veuillez répondre ici à la question.
sentences = []

sentences = load_interest("interest-original.txt")

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

Il y a 2368 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', 'has', 'nt', 'stalled', 'at', 'all', 'mr', 'hines', 'maintains'], ['it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr', 'gorbachev', 'succeed'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', 'expression', 'of', 'interest_1', 'because', 'under', 'chapter', '11', 'revco', 'has', 'exclusivity', 'rights', 'until', 'feb', '28']]


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [10]:
# Veuillez répondre ici à la question.
def wsd_lesk(senses, sentence, window_size=2):
    pos = [i for i, w in enumerate(sentence) if re.match(f"^interest[s]?_[1-{len(senses)}]$", w)][0]
    begin = max(0, pos - window_size)
    end = min(len(sentence) - 1, pos + window_size)

    window_words = sentence[begin:pos] + sentence[pos+1:end+1]
    occ = [len([w for w in window_words if w in sense]) for sense in senses]

    best = max(occ)
    bests = [i for i, o in enumerate(occ) if o == best]
    return random.choice(bests) + 1

**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [11]:
# Veuillez répondre ici à la question.
def evaluate_wsd(fct_name, senses, sentences, window_size=2):
    n_correct = 0
    for sentence in sentences:
        true = int([w for w in sentence if re.match(f"^interest[s]?_[1-{len(senses)}]$", w)][0][-1])
        pred = fct_name(senses, sentence, window_size)

        if true == pred:
            n_correct += 1

    return n_correct / len(sentences)

**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [12]:
# Veuillez répondre ici à la question.
best_a = (0, 0)
best_b = (0, 0)
for window_size in range(2, 30+1):
    a = evaluate_wsd(wsd_lesk, senses1, sentences, window_size=window_size)
    b = evaluate_wsd(wsd_lesk, senses2, sentences, window_size=window_size)

    if a > best_a[0]:
        best_a = a, window_size

    if b > best_b[0]:
        best_b = b, window_size

    print(f"window: {window_size} senses1: {a:.3f} senses2: {b:.3f}")

print(f"best senses1 at {best_a[1]}: {best_a[0]}")
print(f"best senses2 at {best_b[1]}: {best_b[0]}")

window: 2 senses1: 0.166 senses2: 0.176
window: 3 senses1: 0.174 senses2: 0.192
window: 4 senses1: 0.194 senses2: 0.190
window: 5 senses1: 0.191 senses2: 0.206
window: 6 senses1: 0.184 senses2: 0.211
window: 7 senses1: 0.183 senses2: 0.212
window: 8 senses1: 0.176 senses2: 0.232
window: 9 senses1: 0.185 senses2: 0.217
window: 10 senses1: 0.188 senses2: 0.209
window: 11 senses1: 0.187 senses2: 0.216
window: 12 senses1: 0.184 senses2: 0.212
window: 13 senses1: 0.184 senses2: 0.228
window: 14 senses1: 0.181 senses2: 0.225
window: 15 senses1: 0.172 senses2: 0.232
window: 16 senses1: 0.183 senses2: 0.227
window: 17 senses1: 0.186 senses2: 0.218
window: 18 senses1: 0.184 senses2: 0.232
window: 19 senses1: 0.195 senses2: 0.229
window: 20 senses1: 0.182 senses2: 0.237
window: 21 senses1: 0.183 senses2: 0.248
window: 22 senses1: 0.189 senses2: 0.231
window: 23 senses1: 0.174 senses2: 0.240
window: 24 senses1: 0.177 senses2: 0.234
window: 25 senses1: 0.185 senses2: 0.233
window: 26 senses1: 0.17

In [13]:
# Le meilleur résultat est ~24%, obtenu avec la liste `senses2` et un `window_size` de 21

## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [14]:
#import gensim.downloader as api
#api.load("word2vec-google-news-300")

In [15]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "/home/hugo/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [16]:
# Veuillez répondre ici à la question.
def wsd_word2vec(senses, sentence, window_size=2):
    pos = [i for i, w in enumerate(sentence) if re.match(f"^interest[s]?_[1-{len(senses)}]$", w)][0]
    begin = max(0, pos - window_size)
    end = min(len(sentence) - 1, pos + window_size)

    window_words = sentence[begin:pos] + sentence[pos+1:end+1]
    occ = [wv_model.n_similarity(window_words, sense) for sense in senses]

    best = max(occ)
    bests = [i for i, o in enumerate(occ) if o == best]
    return random.choice(bests) + 1

**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [17]:
# Veuillez répondre ici à la question.
best_a = (0, 0)
best_b = (0, 0)
for window_size in range(2, 30+1, 2):
    a = evaluate_wsd(wsd_word2vec, senses1, sentences, window_size=window_size)
    b = evaluate_wsd(wsd_word2vec, senses2, sentences, window_size=window_size)

    if a > best_a[0]:
        best_a = a, window_size

    if b > best_b[0]:
        best_b = b, window_size

    print(f"window: {window_size} senses1: {a:.3f} senses2: {b:.3f}")

print(f"best senses1 at {best_a[1]}: {best_a[0]}")
print(f"best senses2 at {best_b[1]}: {best_b[0]}")

window: 2 senses1: 0.277 senses2: 0.419
window: 4 senses1: 0.305 senses2: 0.423
window: 6 senses1: 0.320 senses2: 0.435
window: 8 senses1: 0.307 senses2: 0.433
window: 10 senses1: 0.320 senses2: 0.436
window: 12 senses1: 0.315 senses2: 0.440
window: 14 senses1: 0.311 senses2: 0.437
window: 16 senses1: 0.307 senses2: 0.436
window: 18 senses1: 0.308 senses2: 0.438
window: 20 senses1: 0.308 senses2: 0.438
window: 22 senses1: 0.306 senses2: 0.435
window: 24 senses1: 0.304 senses2: 0.434
window: 26 senses1: 0.305 senses2: 0.432
window: 28 senses1: 0.304 senses2: 0.432
window: 30 senses1: 0.305 senses2: 0.434
best senses1 at 6: 0.32010135135135137
best senses2 at 12: 0.4396114864864865


In [18]:
# On obtient le meilleur score de ~44% avec `sense2` et une `window_size` de 12, soit un meilleur
# résultat qu'avec l'algorithme de Lesk

## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits (attributs, *features*), pour créer les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

### 4.1. Attributs lexicaux positionnels

Dans cette première représentation des attributs, vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou `NONE` si la fenêtre dépasse l'étendue de la phrase.  Vous ajouterez un attribut qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devrez donc créer une représentation formelle, incluant un dictionnaire Python et le numéro du sens :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
Vous regrouperez toutes ces entrées dans une liste totale appelée `items_with_features_A`.  (Le numéro du sens servira à l'entraînement, puis il sera caché à l'évaluation, quand on comparera la prédiction du système au numéro correct.)  

**4.1a.** En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste totale, en vous aidant si nécessaire du livre NLTK.

In [19]:
def featuresA(sentences, n_senses=6, window_size=2):
    ret = []
    for sentence in sentences:
        pos = [i for i, w in enumerate(sentence) if re.match(f"^interest[s]?_[1-{n_senses}]$", w)][0]

        cur = {}
        for i in range(-window_size, window_size+1):
            match pos + i:
                case x if x < 0: cur[f"word{i}"] = "NONE"
                case x if x >= len(sentence): cur[f"word+{i}"] = "NONE"
                case x if i < 0: cur[f"word{i}"] = sentence[x]
                case x if i > 0: cur[f"word+{i}"] = sentence[x]
                case _: cur["word0"] = sentence[x][:-2]
        ret.append([cur, int(sentence[pos][-1])])
    return ret

In [20]:
# Veuillez répondre ici à la question.
items_with_features_A = featuresA(sentences, window_size=3)

print(len(items_with_features_A))
print(items_with_features_A[151:154])

2368
[[{'word-3': 'NONE', 'word-2': 'NONE', 'word-1': 'investor', 'word0': 'interest', 'word+1': 'in', 'word+2': 'stock', 'word+3': 'funds'}, 1], [{'word-3': 'in', 'word-2': 'the', 'word-1': 'western', 'word0': 'interest', 'word+1': 'to', 'word+2': 'see', 'word+3': 'mr'}, 4], [{'word-3': 'an', 'word-2': 'expression', 'word-1': 'of', 'word0': 'interest', 'word+1': 'because', 'word+2': 'under', 'word+3': 'chapter'}, 1]]


**4.1b.** On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Vous garderez 80% des données pour l'entraînement et utiliserez les 20% restants pour l'évaluation.  Veuillez faire cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ ("stratification"), et enregistrer les deux sous-ensembles de `items_with_features_A` sous les noms respectifs de `iwf_A_train` et `iwf_A_test`.

In [21]:
from random import shuffle

In [22]:
def split_train_test(features, n_senses=6, ratio=0.8):
    train = []
    test  = []
    # Veuillez répondre ici à la question.

    features = features.copy()
    shuffle(features)
    for sense in range(1, len(senses1) + 1):
        sense_items = [items for items in features if items[1] == sense]

        cutoff = int(round(len(sense_items) * ratio))
        train += sense_items[:cutoff]
        test += sense_items[cutoff:]
        
    return train, test

In [23]:
iwf_A_train, iwf_A_test = split_train_test(items_with_features_A, len(senses1))

print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

1895   473
[[{'word-3': 'watching', 'word-2': 'them', 'word-1': 'with', 'word0': 'interest', 'word+1': 'NONE', 'word+2': 'NONE', 'word+3': 'NONE'}, 1], [{'word-3': 'research', 'word-2': 'institute', 'word-1': 'said', 'word0': 'interest', 'word+1': 'in', 'word+2': 'exploration', 'word+3': 'is'}, 1]] [[{'word-3': '700', 'word-2': 'million', 'word-1': 'in', 'word0': 'interest', 'word+1': 'payments', 'word+2': 'a', 'word+3': 'year'}, 6], [{'word-3': 'NONE', 'word-2': 'earnings', 'word-1': 'before', 'word0': 'interest', 'word+1': 'and', 'word+2': 'tax', 'word+3': 'from'}, 6]]


**4.1c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez (avec la fonction `accuracy`), et comment se compare-t-il avec les précédents ?

In [24]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.

best = (0, 0)
for window_size in range(1, 30+1, 2):
    items_with_features = featuresA(sentences, window_size=window_size)
    train, test = split_train_test(items_with_features)

    clf = naivebayes.NaiveBayesClassifier.train(train)
    score = nltk.classify.accuracy(clf, test)

    if score > best[1]:
        best = (window_size, score)
    print(f"window: {window_size} score: {score:.3f}")

print(f"best at {best[0]}: {best[1]}")

window: 1 score: 0.858
window: 3 score: 0.860
window: 5 score: 0.860
window: 7 score: 0.856
window: 9 score: 0.854
window: 11 score: 0.839
window: 13 score: 0.812
window: 15 score: 0.793
window: 17 score: 0.825
window: 19 score: 0.827
window: 21 score: 0.791
window: 23 score: 0.770
window: 25 score: 0.784
window: 27 score: 0.772
window: 29 score: 0.751
best at 3: 0.8604651162790697


In [25]:
# On obtient le meilleur score de loin jusqu'à maintenant, avec ~86% pour une `window_size`
# ayant une faible influence à partir de ~3 et qui empire le résultat à partir d'une valeur d'environ 15

In [26]:
clf = naivebayes.NaiveBayesClassifier.train(iwf_A_train)
print(nltk.classify.accuracy(clf, iwf_A_test))

0.864693446088795


**4.1d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [27]:
clf.show_most_informative_features()

Most Informative Features
                  word+1 = 'in'                1 : 6      =     72.4 : 1.0
                   word0 = 'interests'         3 : 1      =     66.7 : 1.0
                  word-1 = 'other'             3 : 6      =     35.0 : 1.0
                  word+1 = 'of'                4 : 6      =     32.1 : 1.0
                  word-2 = 'have'              1 : 6      =     23.9 : 1.0
                  word-3 = 'NONE'              6 : 3      =     22.1 : 1.0
                  word+2 = 'the'               1 : 3      =     21.3 : 1.0
                  word+2 = 'and'               6 : 5      =     21.2 : 1.0
                  word+2 = 'on'                6 : 5      =     20.8 : 1.0
                  word-1 = 'in'                6 : 5      =     17.7 : 1.0


In [28]:
# On constate que les attributs les plus informatifs sont plutôt ceux qui sont les plus proches

### 4.2 Présence de mots indicateurs

Une deuxième façon d'encoder les attributs lexicaux est de constituer un vocabulaire avec tous les mots qui apparaissent dans tous les voisinages de *interest* et de définir ces mots comme attributs.  Par conséquent, pour chaque occurrence de *interest*, on extrait la valeur de ces attributs sous la forme :
```
[{('rate' : True), ('in' : False), ...}, 1]
```
où *'rate'*, *'in'*, etc., sont les mots du vocabulaire, True/False indiquent leur présence/absence autour de l'occurrence de *interest* qui nous intéresse, et le dernier nombre est le sens, entre 1 et 6.

**4.2a.** Pour commencer, en partant de `sentences` et en fixant la taille de la fenêtre, veuillez constituer la liste de tous les mots observés autour de tous les voisinages de toutes les occurrences de *interest*.

In [29]:
def get_word_list(sentences, n_senses=6, window_size=2):
    ret = []
    for sentence in sentences:
        pos = [i for i, w in enumerate(sentence) if re.match(f"^interest[s]?_[1-{n_senses}]$", w)][0]
        begin = max(0, pos - window_size)
        end = min(len(sentence) - 1, pos + window_size)

        window_words = sentence[begin:pos] + sentence[pos+1:end+1]
        ret += window_words
    return ret

In [30]:
word_list = get_word_list(sentences, window_size=3)
# Veuillez répondre ici à la question.

print(len(word_list))
print(word_list[:50])

13040
['further', 'declines', 'in', 'rates', 'to', 'indicate', 'declining', 'rates', 'because', 'they', 'rises', 'in', 'short-term', 'rates', 'a', '834', '%', 'in', 'this', 'energy-services', 'holding', 'company', 'with', 'in', 'the', 'mechanical', 'be', 'refunded', 'plus', 'curry', 'set', 'the', 'rate', 'on', 'the', 'country', 's', 'own', 'prompted', 'the', 'improvements', 'of', 'principal', 'and', 'is', 'the', 'only', 'to', 'increase', 'its']


**4.2b.** En utilisant par exemple un objet de type `nltk.FreqDist`, veuillez sélectioner les 500 mots les plus fréquents (vous pourrez aussi optimiser ce nombre), dans une liste appelée `vocabulary`.  À votre avis, est-ce une bonne idée d'enlever les *stopwords* de cette liste pour construire les traits ?

In [31]:
def voc_from_word_list(word_list, voc_size=500):
    return list(map(lambda kv: kv[0], sorted(dict(nltk.FreqDist(word_list)).items(), key=lambda kv: kv[1], reverse=True)[:voc_size]))

In [32]:
# Veuillez répondre ici à la question.
vocabulary = voc_from_word_list(word_list)
print(vocabulary[:50])

['in', 'the', 'rates', 'and', 'to', 'of', 'a', 'on', 'rate', 's', 'its', '%', 'payments', 'that', 'are', 'has', 'lower', 'with', 'for', 'an', 'is', 'by', 'have', 'will', 'high', 'at', 'short', 'from', 'company', 'or', 'annual', 'their', 'us', 'higher', 'foreign', 'which', 'as', 'minority', 'said', 'bonds', 'other', 'it', 'income', 'nt', 'short-term', 'but', 'pay', 'because', 'be', 'below']


In [33]:
# Non, ce n'est pas forcément une bonne idée car ils peuvent aider à déterminer l'usage du mot
# `interest` selon le sens. p. ex. `it could be *of* interest *to*`

**4.2c.** Veuillez maintenant créer l'ensemble total de données formatées, en convertissant chaque phrase contenant une occurrence de *interest* à un dictionnaire de traits/valeurs (suivi du numéro du sens), comme exemplifié au début de cette section 4.2.  Cet ensemble sera appelé `items_with_features_B`.

In [34]:
def featuresB(sentences, vocabulary, n_senses=6):
    ret = []
    for sentence in sentences:
        pos = [i for i, w in enumerate(sentence) if re.match(f"^interest[s]?_[1-{n_senses}]$", w)][0]
        cur = {}
        for w in vocabulary:
            cur[w] = w in sentence

        ret.append([cur, int(sentence[pos][-1])])
    return ret

In [35]:
items_with_features_B = featuresB(sentences, vocabulary)
print(len(items_with_features_B))

2368


**4.2d.** Comme dans la section 4.1, veuillez créer maintenant deux sous-ensembles de `items_with_features_B` appelés `iwf_B_train` (80% des items) et `iwf_B_test` (20% des items), avec une sélection aléatoire mais stratifiée.

In [36]:
iwf_B_train, iwf_B_test = split_train_test(items_with_features_B)

print(len(iwf_B_train), ' ', len(iwf_B_test))

1895   473


**4.2e.** Comme dans la section 4.1, veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_B_train` et la tester sur `iwf_B_test`.  En expérimentant avec différentes largeurs de fenêtres et tailles du vocabulaire, quel est le meilleur score que vous obtenez, et comment se compare-t-il avec les précédents ?

In [37]:
best = (0, 0, 0)
for voc_size in (100, 500, 1000):
    vocabulary = voc_from_word_list(get_word_list(sentences, window_size=window_size), voc_size=voc_size)
    for window_size in range(1, 30+1, 2):
        items_with_features = featuresB(sentences, vocabulary)
        train, test = split_train_test(items_with_features)

        clf = naivebayes.NaiveBayesClassifier.train(train)
        score = nltk.classify.accuracy(clf, test)

        if score > best[2]:
            best = (voc_size, window_size, score)
        print(f"voc: {voc_size}: window: {window_size} score: {score:.3f}")

print(f"best at voc: {best[0]} window: {best[1]}: {best[2]}")

voc: 100: window: 1 score: 0.742
voc: 100: window: 3 score: 0.757
voc: 100: window: 5 score: 0.744
voc: 100: window: 7 score: 0.742
voc: 100: window: 9 score: 0.721
voc: 100: window: 11 score: 0.691
voc: 100: window: 13 score: 0.738
voc: 100: window: 15 score: 0.753
voc: 100: window: 17 score: 0.744
voc: 100: window: 19 score: 0.742
voc: 100: window: 21 score: 0.732
voc: 100: window: 23 score: 0.746
voc: 100: window: 25 score: 0.784
voc: 100: window: 27 score: 0.736
voc: 100: window: 29 score: 0.786
voc: 500: window: 1 score: 0.812
voc: 500: window: 3 score: 0.814
voc: 500: window: 5 score: 0.839
voc: 500: window: 7 score: 0.801
voc: 500: window: 9 score: 0.795
voc: 500: window: 11 score: 0.818
voc: 500: window: 13 score: 0.841
voc: 500: window: 15 score: 0.833
voc: 500: window: 17 score: 0.814
voc: 500: window: 19 score: 0.822
voc: 500: window: 21 score: 0.812
voc: 500: window: 23 score: 0.822
voc: 500: window: 25 score: 0.833
voc: 500: window: 27 score: 0.805
voc: 500: window: 29 sco

In [38]:
vocabulary = voc_from_word_list(get_word_list(sentences, window_size=17), voc_size=1000)
items_with_features = featuresB(sentences, vocabulary)
train, test = split_train_test(items_with_features)

clf = naivebayes.NaiveBayesClassifier.train(train)
score = nltk.classify.accuracy(clf, test)
print(score)

0.8625792811839323


**4.2f.** Quels sont les attributs les plus informatifs ?

In [39]:
clf.most_informative_features()

[('broadcasting', True),
 ('resigned', True),
 ('area', True),
 ('computer', True),
 ('washington', True),
 ('rates', True),
 ('pursue', True),
 ('best', True),
 ('personal', True),
 ('oil', True),
 ('m', True),
 ('p', True),
 ('special', True),
 ('session', True),
 ('court', True),
 ('end', True),
 ('especially', True),
 ('find', True),
 ('law', True),
 ('little', True),
 ('next', True),
 ('priced', True),
 ('series', True),
 ('set', True),
 ('we', True),
 ('rate', True),
 ('given', True),
 ('leaving', True),
 ('our', True),
 ('investor', True),
 ('maker', True),
 ('values', True),
 ('products', True),
 ('airline', True),
 ('protecting', True),
 ('contract', True),
 ('director', True),
 ('down', True),
 ('five', True),
 ('probably', True),
 ('officer', True),
 ('expressed', True),
 ('almost', True),
 ('home', True),
 ('political', True),
 ('lower', True),
 ('public', True),
 ('might', True),
 ('francs', True),
 ('customers', True),
 ('protect', True),
 ('could', True),
 ('likely', Tru

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des quatre expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

In [40]:
# Lesk: ~24%
# Word2Vec: ~44%
# Attributs positionnels: ~86%
# Vocabulaire: ~86%

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.